<a href="https://colab.research.google.com/github/muhammad-usama-aleem/CIFAR-10-Classification/blob/main/CIFAR_10_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# use this command to allow growth of the memory and this prevent error.
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# if error is 'Physical error cannot be modified after being initailized', then restart kernel.

# **Load Data**

In [4]:
from tensorflow.keras import datasets

# importing cfiar dataset 
(train_x, train_y), (test_x, test_y) = datasets.cifar10.load_data()

# **Data Preprocessing**



1.   **Normalize data**

In [5]:
train_x = train_x/255.0
test_x = test_x/255.0

# **Model**

In [6]:
from tensorflow.keras import models, layers, losses

model = models.Sequential()
# 
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
# softmax for multiclass classification
# sigmoid for binary classification

In [7]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [9]:
model.fit(
    train_x, train_y, epochs=10, validation_data = (test_x, test_y)
)

Epoch 1/10
1563/1563 [==============================] - ETA: 0s - loss: 1.3861 - accuracy: 0.5017

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1563/1563 [==============================] - 13s 8ms/step - loss: 1.3861 - accuracy: 0.5017 - val_loss: 1.1332 - val_accuracy: 0.5922
Epoch 2/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.0219 - accuracy: 0.6401 - val_loss: 0.9682 - val_accuracy: 0.6580
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.8543 - accuracy: 0.7006 - val_loss: 0.9546 - val_accuracy: 0.6736
Epoch 4/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7225 - accuracy: 0.7445 - val_loss: 0.9360 - val_accuracy: 0.6793
Epoch 5/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.6051 - accuracy: 0.7864 - val_loss: 1.0007 - val_accuracy: 0.6684
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.4961 - accuracy: 0.8250 - val_loss: 1.0192 - val_accuracy: 0.6832
Epoch 7/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.3928 - accuracy: 0.8624 - val_loss: 1.1871 - val_accuracy: 0.6

In [10]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 4ms/step - loss: 1.6106 - accuracy: 0.6708


[1.6105700731277466, 0.670799970626831]